In [1]:
from dlcliche.notebook import *
from dlcliche.utils import *
sys.path.append('..')
import img_hash
from utils import *

dataroot = '/data/mvtec_ad/original'

params = EasyDict()
params.suffix = '.png'
params.project = 'mvtecad_phash32'
params.work_folder = 'tmp'

params.max_fpr = 0.1
params.min_tpr = 1.0
params.sigma_k = 2.0

params.data = {}
params.data.pre_crop_rect = None

params.hash_algorithm = 'phash'
params.hash_size = 32

In [2]:
from utils import evaluate_MVTecAD

results = evaluate_MVTecAD(dataroot, img_hash.ImgHashAnoDet, params,
                test_targets=None
                )

Evaluating: ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']

--- Start evaluating [bottle] ----
 preprocessing...
 training...
 using cached tmp/mvtecad_phash32/hashtable-bottle.pkl
 evaluating...
distance_threshold: 1.4427268158992723 # Threshold k-sigma [2.0]
distance_threshold: 0.9847621263231361 # Threshold max FPR [0.1]
distance_threshold: 0.5503082470629289 # Threshold min TPR [1.0]
distance_norm_factor: 0.20230139307228914
Results: {'target': ['bottle'], 'auc': [0.7718253968253967], 'th_k_sigma': [1.4427268158992723], 'th_fpr': [0.9847621263231361], 'th_tpr': [0.5503082470629289], 'norm_factor': [0.20230139307228914], 'pauc': [0.8015873015873016]}

--- Start evaluating [cable] ----
 preprocessing...
 training...
 using cached tmp/mvtecad_phash32/hashtable-cable.pkl
 evaluating...
distance_threshold: 1.1607946880230373 # Threshold k-sigma [2.0]
distance_threshold: 1.0811148

distance_threshold: 1.0788482971364557 # Threshold k-sigma [2.0]
distance_threshold: 1.028903749247298 # Threshold max FPR [0.1]
distance_threshold: 0.9188349760720058 # Threshold min TPR [1.0]
distance_norm_factor: 0.40812551738410596
Results: {'target': ['zipper'], 'auc': [0.7941176470588235], 'th_k_sigma': [1.0788482971364557], 'th_fpr': [1.028903749247298], 'th_tpr': [0.9188349760720058], 'norm_factor': [0.40812551738410596], 'pauc': [0.585858027421495]}
done.


In [3]:
results.to_csv(f'sample_{params.project}.csv')
results

,target,auc,th_k_sigma,th_fpr,th_tpr,norm_factor,pauc
0,bottle,0.771825,1.442727,0.984762,0.550308,0.202301,0.801587
1,cable,0.720109,1.160795,1.081115,0.774903,0.312539,0.549377
2,capsule,0.767252,1.690502,1.008455,0.591627,0.145256,0.659438
3,carpet,0.516051,1.032725,1.018148,0.931127,0.448885,0.588409
4,grid,0.839181,1.239263,1.033292,0.822305,0.361028,0.747186
5,hazelnut,0.843750,1.167915,1.042909,0.849379,0.363317,0.682331
6,leather,0.909477,1.047004,1.004666,0.959815,0.435468,0.783109
7,metal_nut,0.596530,1.678277,1.378152,0.567920,0.257931,0.609508
8,pill,0.614839,1.133876,1.085787,0.859581,0.259029,0.524995
9,screw,0.686719,1.602674,1.248412,0.512914,0.201819,0.550393


## If you want to try one by one

In [4]:
from dlcliche.utils import (load_pkl, save_as_pkl_binary)
sys.path.append('..')
from base_ano_det import BaseAnoDet


test_target = 'transistor'
folder = Path(dataroot)/test_target
good_files = sorted(folder.glob(f'train/good/*{params.suffix}'))
test_files = sorted(folder.glob(f'test/*/*{params.suffix}'))
test_y_trues = [f.parent.name != 'good' for f in test_files]

params.norm_factor = 0.263867 # from test result

det = img_hash.ImgHashAnoDet(params)
det.prepare_experiment()
# no need: det.setup_train(good_files)
det.train_model(good_files)

scores = det.predict(test_files)
norm_scores = det.normalize_score(scores)
print(norm_scores)

 using cached tmp/mvtecad_phash32/hashtable-None.pkl


ValueError: Set params.distance_norm_factor to normalize score.

## If you run for many parameter sets, summarize...

In [25]:
def read_result(file_name, title):
    df = pd.read_csv(file_name)[['target', 'auc']].set_index('target')
    df.columns = [title]
    return df

dfs = [read_result(f, title) for f, title in [
    ('sample_mvtecad_phash16.csv', 'phash=16'),
    ('sample_mvtecad_phash.csv', 'phash=32'),
    ('sample_mvtecad_phash64.csv', 'phash=64'),
    ('sample_mvtecad_whash16.csv', 'whash=16'),
    ('sample_mvtecad_whash.csv', 'whash=32'),
]]

df = pd.concat(dfs, axis=1)
df

,phash=16,phash=32,phash=64,whash=16,whash=32
target,,,,,
bottle,0.894048,0.769048,0.834921,0.831746,0.835714
cable,0.763681,0.717860,0.655454,0.688999,0.756747
capsule,0.732948,0.764061,0.751296,0.540088,0.713801
carpet,0.549559,0.476124,0.493780,0.600120,0.461276
grid,0.913952,0.839181,0.786967,0.619883,0.563910
hazelnut,0.786250,0.836250,0.812500,0.462679,0.530714
leather,0.946671,0.897079,0.757812,0.838655,0.793648
metal_nut,0.660557,0.597996,0.556452,0.714809,0.782014
pill,0.649345,0.628205,0.635706,0.648663,0.854883


In [26]:
print(df.to_markdown())

| target     |   phash=16 |   phash=32 |   phash=64 |   whash=16 |   whash=32 |
|:-----------|-----------:|-----------:|-----------:|-----------:|-----------:|
| bottle     |   0.894048 |   0.769048 |   0.834921 |   0.831746 |   0.835714 |
| cable      |   0.763681 |   0.71786  |   0.655454 |   0.688999 |   0.756747 |
| capsule    |   0.732948 |   0.764061 |   0.751296 |   0.540088 |   0.713801 |
| carpet     |   0.549559 |   0.476124 |   0.49378  |   0.60012  |   0.461276 |
| grid       |   0.913952 |   0.839181 |   0.786967 |   0.619883 |   0.56391  |
| hazelnut   |   0.78625  |   0.83625  |   0.8125   |   0.462679 |   0.530714 |
| leather    |   0.946671 |   0.897079 |   0.757812 |   0.838655 |   0.793648 |
| metal_nut  |   0.660557 |   0.597996 |   0.556452 |   0.714809 |   0.782014 |
| pill       |   0.649345 |   0.628205 |   0.635706 |   0.648663 |   0.854883 |
| screw      |   0.598176 |   0.686719 |   0.647469 |   0.573786 |   0.605759 |
| tile       |   0.440657 |   0.478896 |